### Adjacent List Model


(setup)

```
docker run --name my-postgres -e POSTGRES_PASSWORD=mysecretpassword -p 5432:5432 -d postgres
```

In [5]:
import psycopg2
import textwrap
import contextlib

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="postgres",      # default DB
    user="postgres",          # default user
    password="mysecretpassword"
)

def setup():
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS parent_child_adjent_list")
    cur.execute(textwrap.dedent("""
        CREATE TABLE parent_child_adjent_list (
            id INT,
            first_name TEXT,
            last_name TEXT,
            parent_id INT
        );
    """))
    cur.close()
    conn.commit()

@contextlib.contextmanager
def session():
    cur = conn.cursor()
    try:
        cur.execute("DELETE FROM parent_child_adjent_list")
        cur.execute(textwrap.dedent("""
            INSERT INTO parent_child_adjent_list (id, first_name, last_name, parent_id) VALUES
                (1, 'Rosa', 'Wellington', NULL), (2, 'Jon', 'Wellington', 1), (3, 'Joni', 'Wellington', 1),
                (4, 'Marge', 'Wellington', 1), (5, 'Mary', 'Dijkstra', 2), (6, 'Frank', 'Wellington', 2),
                (7, 'Jason', 'Wellington', 3), (8, 'Bobby', 'Wellington', 4), (9, 'Sammy', 'Wellington', 4),
                (10, 'Sarah', 'Wellington', 4), (11, 'Sam Francis', 'Dijkstra', 5), (12, 'Stephen', 'Wellington', 6),
                (13, 'Trent', 'Wellington', 6), (14, 'June', 'Wellington', 9), (15, 'Josephine', 'Wellington', 9),
                (16, 'Suzy', 'Wellington', 9);
        """))
        yield cur
    except Exception:
        conn.rollback()
        raise
    finally:
        cur.close()

def run(cur, sql):
    cur.execute(sql)
    return cur.fetchall()

setup()

##### Find one ancestors

e.g., for each node, find its parent node.

In [7]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        SELECT 
            child.first_name,
            child.last_name,
            parent.first_name,
            parent.last_name
        FROM parent_child_adjent_list AS parent
        JOIN parent_child_adjent_list AS child ON parent.id = child.parent_id
    """)))

[('Marge', 'Wellington', 'Rosa', 'Wellington'), ('Joni', 'Wellington', 'Rosa', 'Wellington'), ('Jon', 'Wellington', 'Rosa', 'Wellington'), ('Frank', 'Wellington', 'Jon', 'Wellington'), ('Mary', 'Dijkstra', 'Jon', 'Wellington'), ('Jason', 'Wellington', 'Joni', 'Wellington'), ('Sarah', 'Wellington', 'Marge', 'Wellington'), ('Sammy', 'Wellington', 'Marge', 'Wellington'), ('Bobby', 'Wellington', 'Marge', 'Wellington'), ('Sam Francis', 'Dijkstra', 'Mary', 'Dijkstra'), ('Trent', 'Wellington', 'Frank', 'Wellington'), ('Stephen', 'Wellington', 'Frank', 'Wellington'), ('Suzy', 'Wellington', 'Sammy', 'Wellington'), ('Josephine', 'Wellington', 'Sammy', 'Wellington'), ('June', 'Wellington', 'Sammy', 'Wellington')]


##### Find one descendents

...

##### Depth/Level

Recursive CTE(Common Table Expression)

e.g., get a generation number for each node


In [ ]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        WITH RECURSIVE generation AS (
            SELECT 
                id, first_name, last_name, 1 AS generation_number 
            FROM parent_child_adjent_list 
            WHERE parent_id IS NULL
            UNION ALL
            SELECT 
                child.id, child.first_name, child.last_name, g.generation_number + 1 AS generation_number
            FROM parent_child_adjent_list child
            JOIN generation g ON child.parent_id = g.id
        ) SELECT * FROM generation;
    """)))


[(1, 'Rosa', 'Wellington', 1), (2, 'Jon', 'Wellington', 2), (3, 'Joni', 'Wellington', 2), (4, 'Marge', 'Wellington', 2), (5, 'Mary', 'Dijkstra', 3), (6, 'Frank', 'Wellington', 3), (7, 'Jason', 'Wellington', 3), (8, 'Bobby', 'Wellington', 3), (9, 'Sammy', 'Wellington', 3), (10, 'Sarah', 'Wellington', 3), (11, 'Sam Francis', 'Dijkstra', 4), (12, 'Stephen', 'Wellington', 4), (13, 'Trent', 'Wellington', 4), (14, 'June', 'Wellington', 4), (15, 'Josephine', 'Wellington', 4), (16, 'Suzy', 'Wellington', 4)]


### Modified Preorder Tree Traversal(MPTT)

Represent tree with a **nested-set data structure**

* assign a range to each node: (left, right)
* parent's range is a superset of its children ranges

For example, to represent the hierarchy in an organization,

|  Name  | left | right |
| ------ | ---- | ----- |
| Albert |   1  |  12   |
| Bert   |   2  |  3    |
| Chuck  |   4  |  11   |
| Donna  |   5  |  6    |
| Eddie  |   7  |  8    |
| Fred   |   9  |  10   |

![Tree](https://www.ibase.ru/files/articles/programming/dbmstrees/Celko.10.19.Fig1s.gif)

Performance
* Efficient retrieval operations
* Trade-off: inserts, update
 

(setup)

```
docker run --name my-postgres -e POSTGRES_PASSWORD=mysecretpassword -p 5432:5432 -d postgres
```

In [ ]:
import psycopg2
import textwrap
import contextlib

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="postgres",      # default DB
    user="postgres",          # default user
    password="mysecretpassword"
)

def setup():
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS personnel_mptt")
    cur.execute(textwrap.dedent("""
        CREATE TABLE personnel_mptt (
            name TEXT,
            vleft INT,
            vright INT
        );
    """))
    cur.execute("DROP TABLE IF EXISTS salary_mptt")
    cur.execute(textwrap.dedent("""
        CREATE TABLE salary_mptt (
            name TEXT,
            amount INT
        );
    """))
    cur.close()
    conn.commit()

@contextlib.contextmanager
def session():
    cur = conn.cursor()
    try:
        cur.execute("DELETE FROM personnel_mptt")
        cur.execute(textwrap.dedent("""
            INSERT INTO personnel_mptt (name, vleft, vright) VALUES
                ('Albert', 1, 12), ('Bert', 2, 3), ('Chuck', 4, 11),
                ('Donna', 5, 6), ('Eddie', 7, 8), ('Fred', 9, 10);
        """))
        cur.execute("DELETE FROM salary_mptt")
        cur.execute(textwrap.dedent("""
            INSERT INTO salary_mptt (name, amount) VALUES
                ('Albert', 100), ('Bert', 85), ('Chuck', 70),
                ('Donna', 60), ('Eddie', 65), ('Fred', 50);
        """))
        yield cur
    except Exception:
        conn.rollback()
        raise
    finally:
        cur.close()

def run(cur, sql):
    cur.execute(sql)
    return cur.fetchall()

setup()


##### Find one/all ancestors

e.g., find Eddie and all his supervisors, no matter how deep the tree.

In [3]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        SELECT p2.*
        FROM personnel_mptt AS p1, personnel_mptt AS p2
        WHERE p1.name = 'Eddie'
            AND p1.vleft BETWEEN p2.vleft AND p2.vright
    """)))

[('Albert', 1, 12), ('Chuck', 4, 11), ('Eddie', 7, 8)]


##### Find one/all descendants

Find the Chuck and all his subordinates.

In [4]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        SELECT p2.*
        FROM personnel_mptt AS p1, personnel_mptt AS p2
        WHERE p1.name = 'Chuck'
            AND p2.vleft BETWEEN p1.vleft AND p1.vright
    """)))

[('Chuck', 4, 11), ('Donna', 5, 6), ('Eddie', 7, 8), ('Fred', 9, 10)]


##### Aggregate

Calculate the total salaries that each employee controls.

In [5]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        SELECT p1.name, SUM(s.amount)
        FROM personnel_mptt AS p1, personnel_mptt AS p2
        JOIN salary_mptt AS s ON p2.name = s.name
        WHERE p2.vleft BETWEEN p1.vleft AND p1.vright
        GROUP BY p1.name
    """)))

[('Eddie', 65), ('Chuck', 245), ('Bert', 85), ('Donna', 60), ('Fred', 50), ('Albert', 430)]


##### Depth/Level

Find the level of each node and order by levels, so you can print the tree as an indented listing.

Similar to find all ancestor, but count ancestors.

In [25]:
with session() as cursor:
    print(run(cursor, textwrap.dedent("""
        SELECT p1.name, COUNT(p2.*) AS level
        FROM personnel_mptt AS p1, personnel_mptt AS p2
        WHERE p1.vleft BETWEEN p2.vleft AND p2.vright
        GROUP BY p1.name, p1.vleft
        ORDER BY p1.vleft
    """)))

[('Albert', 1), ('Bert', 2), ('Chuck', 2), ('Donna', 3), ('Eddie', 3), ('Fred', 3)]


##### Insert 

The nested set model implies an ordering of siblings. Insert a new node Brian under Albert and between Bert and Chuck.

1. Find Brian's rightmost sibling: Bert `(2, 3)`
2. Assign Brian with range `(4, 5)`
3. Update left/right for his right siblings and their descendants: Chuck and ...
4. Update right for his ancestors: Albert, ...


In [44]:
with session() as cursor:
    cursor.execute(textwrap.dedent("""
        DO $$
        DECLARE 
            rightmost_sibling INTEGER;
        BEGIN
            SELECT vright INTO rightmost_sibling FROM personnel_mptt WHERE name = 'Bert';
            
            UPDATE personnel_mptt
            SET vleft = CASE WHEN vleft >= rightmost_sibling + 1 THEN vleft + 2
                            ELSE vleft END,
                vright = CASE WHEN vright >= rightmost_sibling + 2 THEN vright + 2
                            ELSE vright END
            WHERE vright >= rightmost_sibling + 1;

            INSERT INTO personnel_mptt (name, vleft, vright)
            VALUES ('Brian', rightmost_sibling + 1, rightmost_sibling + 2);
        END $$
    """))

    print(run(cursor, "SELECT * FROM personnel_mptt ORDER BY vleft"))

[('Albert', 1, 14), ('Bert', 2, 3), ('Brian', 4, 5), ('Chuck', 6, 13), ('Donna', 7, 8), ('Eddie', 9, 10), ('Fred', 11, 12)]
